<a href="https://colab.research.google.com/github/aida-am247/Deep-Learning/blob/master/E1/DL_E1_callback_stopTraining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
!wget https://raw.githubusercontent.com/Alireza-Akhavan/SRU-deeplearning-workshop/master/dataset.py
!mkdir dataset
!wget https://github.com/Alireza-Akhavan/SRU-deeplearning-workshop/raw/master/dataset/Data_hoda_full.mat -P dataset

--2020-09-22 04:50:26--  https://raw.githubusercontent.com/Alireza-Akhavan/SRU-deeplearning-workshop/master/dataset.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 929 [text/plain]
Saving to: ‘dataset.py’

dataset.py          100%[===================>]     929  --.-KB/s    in 0s      

2020-09-22 04:50:26 (51.3 MB/s) - ‘dataset.py’ saved [929/929]

--2020-09-22 04:50:27--  https://github.com/Alireza-Akhavan/SRU-deeplearning-workshop/raw/master/dataset/Data_hoda_full.mat
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Alireza-Akhavan/SRU-deeplearning-workshop/master/dataset/Data_hoda_full.mat [foll

In [38]:
import keras
from keras.callbacks import Callback
from keras.models import Sequential
from keras.layers import Flatten, Dense, Conv2D, MaxPooling2D, Dropout
import numpy as np
from dataset import load_hoda
import matplotlib.pyplot as plt

In [39]:
class my_callback(Callback):
  def on_epoch_end(self, epoch, logs={}):
    print(logs)
    if logs.get('accuracy') > 0.75:
      print("\nReached 75% accuracy so cancelling training!")
      self.model.stop_training = True
callback = my_callback()

In [40]:
mnist = tf.keras.datasets.fashion_mnist
(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

np.random.seed(123)  # for reproducibility

# Load pre-shuffled HODA data into train and test sets
x_train_original, y_train_original, x_test_original, y_test_original = load_hoda(
                                                                        training_sample_size=3500,                                                                      test_sample_size=400,size=28)

# Preprocess input data
''' 3.1: input data in numpy array format'''
x_train = np.array(x_train_original)
x_test = np.array(x_test_original)
'''3.2 normalize our data values to the range [0, 1]'''
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# Reshape to original image shape (n x 784)  ==> (n x 28 x 28 x 1)
x_train = x_train.reshape(-1,28,28,1)
x_test = x_test.reshape(-1,28,28,1)


# Preprocess class labels
y_train = keras.utils.to_categorical(y_train_original, num_classes=10)
y_test = keras.utils.to_categorical(y_test_original, num_classes=10)


# test and validation set
x_val = x_test[:200]
x_test = x_test[200:]
y_val = y_test[:200]
y_test = y_test[200:]

In [41]:
# Define model architecture
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu',
                        input_shape=(28, 28, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))


# Compile model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [42]:
# Fit model on training data
history = model.fit(x_train, y_train,
          epochs=200, batch_size=256, validation_data = (x_val, y_val), callbacks=[callback])

Epoch 1/200
14/14 [==============================] - 0s 18ms/step - loss: 2.0528 - accuracy: 0.3031 - val_loss: 1.3656 - val_accuracy: 0.7100
Epoch 2/200
14/14 [==============================] - 0s 9ms/step - loss: 1.1969 - accuracy: 0.5949 - val_loss: 0.6065 - val_accuracy: 0.8150
Epoch 3/200
14/14 [==============================] - 0s 9ms/step - loss: 0.7949 - accuracy: 0.7377 - val_loss: 0.3789 - val_accuracy: 0.8850
Epoch 4/200
14/14 [==============================] - ETA: 0s - loss: 0.5995 - accuracy: 0.7986{'loss': 0.5995050072669983, 'accuracy': 0.7985714077949524, 'val_loss': 0.2681453227996826, 'val_accuracy': 0.9150000214576721}

Reached 75% accuracy so cancelling training!
14/14 [==============================] - 0s 9ms/step - loss: 0.5995 - accuracy: 0.7986 - val_loss: 0.2681 - val_accuracy: 0.9150
